In [12]:
import pandas as pd
import numpy as np
import datetime

from OddsCalculator import OddsCalculator as util

In [13]:
#Raw odds from two sources
# 1. https://www.sportsbettingdime.com/politics/2020-us-presidential-election-odds/
# 2. https://www.gamblingsites.org/blog/2020-election-betting-odds-election-night-changes/

odds = pd.read_excel('Election Odds Raw.xlsx', sheet_name='Sheet1').set_index('date')

In [14]:
odds

,trump_odds,biden_odds,source
date,,,
2020-04-08,-121,114,1
2020-04-09,-121,114,1
2020-04-10,-121,114,1
2020-04-11,-121,114,1
2020-04-12,-121,114,1
2020-04-13,-121,114,1
2020-04-14,-121,114,1
2020-04-15,-116,110,1
2020-04-16,-116,110,1


In [15]:
odds['trump_odds'] = odds['trump_odds'].astype(str)
odds['biden_odds'] = odds['biden_odds'].astype(str)

In [16]:
#adds a plus sign to the American odds. Essential for conversion to probabilities

def add_plus(val):
    if '-' not in val and '+' not in val:
        val = '+' + val
    return val

In [17]:
odds['trump_odds'] = odds['trump_odds'].apply(add_plus)
odds['biden_odds'] = odds['biden_odds'].apply(add_plus)

In [18]:
odds['trump_odds_dec'] = odds['trump_odds'].apply(util.calculate_odds).str[1]
odds['biden_odds_dec'] = odds['biden_odds'].apply(util.calculate_odds).str[1]

In [19]:
#Calculate the book's hold, for fun

odds['hold'] = odds.apply(lambda x: util.calculate_hold([x['trump_odds_dec'], x['biden_odds_dec']]),axis=1)

In [20]:
#Calculate the true odds of each candidate winning

odds['trump_win_perc'] = odds.apply(lambda x: util.actual_probability([x['trump_odds'], x['biden_odds_dec']]),axis=1).str[0]
odds['biden_win_perc'] = odds.apply(lambda x: util.actual_probability([x['trump_odds'], x['biden_odds_dec']]),axis=1).str[1]

In [56]:
#Append a final row indicating Biden's win

final_date = pd.Timestamp(2020,11,7)
final_trump_win_perc = 0.0
final_biden_win_perc = 1.0

d = {'date': [final_date], 'trump_win_perc': [final_trump_win_perc], 'biden_win_perc': [final_biden_win_perc]}
final_row = pd.DataFrame(data=d).set_index('date')

In [58]:
final_row

,trump_win_perc,biden_win_perc
date,,
2020-11-07,0.0,1.0


In [59]:
odds = odds.append(final_row, sort=False)

In [62]:
odds.to_csv('final_odds.csv')